In [1]:
import ee
import datetime


In [4]:
ee.Authenticate()

In [6]:
ee.Initialize(project='ee-oosterhuisthijs')

In [11]:


# Define the polygon (adjust coordinates to match your region of interest)
polygon = ee.Geometry.Rectangle([ 4.73 , 52.095, 4.878 ,  52.182  ])

# Function to filter images for RGB and specific season
def get_seasonal_image(start_date, end_date, polygon, season, year):
    # Load Sentinel-2 collection and filter for date, bounds, and cloud percentage
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(polygon) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE')

    # Get the first image and select only the RGB bands
    image = collection.first().select(['TCI_R', 'TCI_G', 'TCI_B'])

    # Add the date to the filename
    date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    filename = f'Nieuwkoopse_plassen_{season}_{year}_{date_str}'

    # Export the image to Google Drive as a GeoTIFF
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'exported_{season}_{year}_rgb_image',
        folder='earth_engine_exports',
        fileNamePrefix=filename,
        region=polygon,
        scale=10,  # Sentinel-2 RGB bands have 10m resolution
        fileFormat='GeoTIFF',
        crs='EPSG:4326'  # Optional: Set CRS to WGS84
    )
    task.start()
    print(f"Started exporting {season} RGB image for {year}: {filename}")

# Define the date ranges for summer (June-August) and winter (December-February)
def get_summer_winter_images(year, polygon):
    # Summer: June 21 - September 21 (adjust as necessary)
    summer_start = f'{year}-06-21'
    summer_end = f'{year}-09-21'

    # Winter: December 21 - March 21 of the next year (adjust as necessary)
    winter_start = f'{year}-12-21'
    winter_end = f'{year+1}-03-21'

    # Retrieve cloud-free RGB images for both seasons
    get_seasonal_image(summer_start, summer_end, polygon, 'summer', year)
    get_seasonal_image(winter_start, winter_end, polygon, 'winter', year)

# Run the process for each year from 2015 to 2024
for year in range(2018, 2024):
    get_summer_winter_images(year, polygon)

print("All export tasks for RGB images started.")


Started exporting summer RGB image for 2018: Nieuwkoopse_plassen_summer_2018_2018-07-02
Started exporting winter RGB image for 2018: Nieuwkoopse_plassen_winter_2018_2019-02-27
Started exporting summer RGB image for 2019: Nieuwkoopse_plassen_summer_2019_2019-08-26
Started exporting winter RGB image for 2019: Nieuwkoopse_plassen_winter_2019_2020-02-07
Started exporting summer RGB image for 2020: Nieuwkoopse_plassen_summer_2020_2020-07-31
Started exporting winter RGB image for 2020: Nieuwkoopse_plassen_winter_2020_2021-01-25
Started exporting summer RGB image for 2021: Nieuwkoopse_plassen_summer_2021_2021-07-21
Started exporting winter RGB image for 2021: Nieuwkoopse_plassen_winter_2021_2022-03-08
Started exporting summer RGB image for 2022: Nieuwkoopse_plassen_summer_2022_2022-07-19
Started exporting winter RGB image for 2022: Nieuwkoopse_plassen_winter_2022_2023-02-14
Started exporting summer RGB image for 2023: Nieuwkoopse_plassen_summer_2023_2023-09-07
Started exporting winter RGB ima